In [1]:
import pandas as pd
from geopy import distance
from tqdm.auto import tqdm

In [2]:
tqdm.pandas()
gardiner_count = pd.read_csv("../../raw_data/gardiner_count_data_v2.zip")
gardiner_count_sorted_by_date = gardiner_count.sort_values(by=["datetime_bin"])
gardiner_count_sorted_by_date[["latitude", "longitude"]] = gardiner_count[["latitude", "longitude"]].progress_apply(
    pd.to_numeric)

  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
available_dates = gardiner_count_sorted_by_date["datetime_bin"].drop_duplicates().to_frame()
print(len(available_dates))
# 7058 unique dates and times

7058


In [4]:
gardiner_count_grouped = gardiner_count_sorted_by_date.groupby(["detector_loc", "datetime_bin"]).size()
gardiner_count_grouped.to_csv("../../csv/gardiner_count_grouped_v2.csv.zip", index=False, compression="zip")
gardiner_count_grouped = pd.read_csv("../../csv/gardiner_count_grouped_v2.csv.zip")

In [5]:
TOMTOM_GARDINER_POINT = (43.642355, -79.379665)
tqdm.pandas()
gardiner_count_sorted_by_date["distance_to_union_km"] = gardiner_count_sorted_by_date \
    .progress_apply(lambda row: distance.distance((row["latitude"], row["longitude"]),
                                                  TOMTOM_GARDINER_POINT).km, axis=1)
gardiner_count_sorted_by_date.to_csv(
    "../../csv/gardiner_count_with_distance_v2.csv.zip", index=False, compression="zip")

  0%|          | 0/145099 [00:00<?, ?it/s]

In [6]:
# get detector locations up to 5km away from union
entries_of_interest = gardiner_count_sorted_by_date[gardiner_count_sorted_by_date["distance_to_union_km"] <= 5]
entries_of_interest = entries_of_interest.sort_values(by=["datetime_bin", "distance_to_union_km"])
entries_of_interest.to_csv("../../csv/gardiner_count_entries_of_interest_v2.csv.zip", index=False, compression="zip")

In [7]:
# spadina has data from August 11 to December 31
spadina_detectors = entries_of_interest[entries_of_interest["detector_loc"].str.contains("SPADINA")]
spadina_detectors.to_csv("../../csv/gardiner_spadina_count_v2.csv.zip", index=False, compression="zip")